# TPGshipのシミュレーション動画作成プログラム（1/4）

## 運用動作画像作成編

ライブラリ読み込み

In [1]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import polars as pl
import numpy as np
import math
from numpy import linalg as LA
import cartopy.feature as cfeature
from PIL import Image
import matplotlib.patches as patches
from datetime import datetime, timedelta, timezone
from dateutil import tz
import os


データファイル読み取り

台風＆TPGship経路データ

In [2]:
year = 2018

TPGship_data = pl.read_csv("output_Folder/TPGshipdata/" + str(year) + "_sim_operation_data+wind.csv")

typhoon_data = pl.read_csv("typhoon_data_6hour_intervals_verpl/table" + str(year) + "_6_interval.csv",encoding="shift-jis")

拠点と運搬船

In [3]:
stBASE_data = pl.read_csv("output_Folder/storageBASEdata/" + str(year) + "storageBASE.csv")
spSHIP1_data = pl.read_csv("output_Folder/supportSHIPdata/" + str(year) + "supportSHIP1.csv")
spSHIP2_data = pl.read_csv("output_Folder/supportSHIPdata/" + str(year) + "supportSHIP2.csv") 

TPGshipの動作画像化

In [4]:
UTC = timezone(timedelta(hours =+ 0),"UTC")
month = 1
wind_data = pl.read_csv("ERA5/wind_datas/era5_testdata_E180W90S0W90_" + str(int(year)) + "_" + str(int(month)) + ".csv",encoding="shift-jis")
wind_lon = wind_data[:]['LON'] # 経度データ読み込み
wind_lat = wind_data[:]['LAT'] # 緯度データ読み込み
wind_u = wind_data[:]["U10_E+_W-[m/s]"]
wind_v = wind_data[:]["V10_N+_S-[m/s]"]
#LONとLATが偶数の時のみのデータを抽出
wind_data = wind_data.filter(wind_data['LON'] % 2 == 0)
wind_data = wind_data.filter(wind_data['LAT'] % 2 == 0)
# グラフ保存用のフォルダがなければ作成
os.makedirs("png_Folder2", exist_ok=True)

for j in range(len(TPGship_data)):
#for j in range(4):
#    j_ori = j
#    j = j + 450
    current_time = TPGship_data[j,"unixtime"]
    if month != datetime.fromtimestamp(current_time,UTC).month:
        month = datetime.fromtimestamp(current_time,UTC).month
        wind_data = pl.read_csv("ERA5/wind_datas/era5_testdata_E180W90S0W90_" + str(int(year)) + "_" + str(int(month)) + ".csv",encoding="shift-jis")
        #LONとLATが偶数の時のみのデータを抽出
        wind_data = wind_data.filter(wind_data['LON'] % 2 == 0)
        wind_data = wind_data.filter(wind_data['LAT'] % 2 == 0)
        wind_lon = wind_data[:]['LON'] # 経度データ読み込み
        wind_lat = wind_data[:]['LAT'] # 緯度データ読み込み
        wind_u = wind_data[:]["U10_E+_W-[m/s]"]
        wind_v = wind_data[:]["V10_N+_S-[m/s]"]

    #地図の作成
    fig = plt.figure(figsize=(12,16)) # プロット領域の作成（matplotlib）
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree()) # サブプロット作成時にcartopy呼び出し

    ax.set_facecolor('paleturquoise')
    land_h = cfeature.NaturalEarthFeature('physical', 'land', '50m')
    ax.add_feature(land_h,color='g')
    ax.set_extent([120, 180, 0, 70], ccrs.Geodetic())


    for i in range(len(wind_lat)):
        if not math.isnan(wind_u[i]):
            #風矢印表示
            size = 12
            vec_size = (np.sqrt((wind_u[i])**2 + (wind_v[i])**2))
            Q_wind = ax.quiver(wind_lon[i], wind_lat[i], wind_u[i]/vec_size*size, wind_v[i]/vec_size*size, vec_size,cmap = 'YlOrRd', clim = (0,15), width=.003, scale=400.)
            


    

    #中継貯蔵拠点&待機位置
    base_lat = 24
    base_lon = 153
    ax.plot(base_lon,base_lat,'crimson',markersize=15,marker="d")

    #供給拠点
    spbase_lat = 34.75
    spbase_lon = 134.79
    ax.plot(spbase_lon,spbase_lat,'crimson',markersize=20,marker="*")



    #陸地境界の設定
    #nonaggression_line_lat = [0,13,13,15,15,24,24,26,26,28,28,32.2,32.2,34,34,41.2,41.2,44,44,50,50]
    #nonaggression_line_lon = [127.5,127.5,125,125,123,123,126,126,130.1,130.1,132.4,132.4,137.2,137.2,143,143,149,149,156,156,180]
    #for i in range(len(nonaggression_line_lat)-1):
    #    ax.plot([nonaggression_line_lon[i],nonaggression_line_lon[i+1]],[nonaggression_line_lat[i],nonaggression_line_lat[i+1]],'red',linewidth=4)

    #台風の作成 
    unixtime = TPGship_data[j,"unixtime"]
    
    TYdata = typhoon_data.filter((pl.col("unixtime") == unixtime))
    typhoon_lon = TYdata[:]["LON"]
    typhoon_lat = TYdata[:]["LAT"]
    if len(typhoon_lon) != 0:
        for k in range(len(typhoon_lon)):
            ax.plot(typhoon_lon[k],typhoon_lat[k],'grey',markersize=50,marker="o",alpha=0.8)


    #凡例の作成
    view_lon = 170
    text_lon = 173
    d_view_lat = 3
    view_ship_lon = 150
    view_ship_lat = 60
    view_lat_1 = 58
    lon_width = 2

    ship_text_lon = 154
    ship_text_lat = 63.5

    #目標名の設定
    target = TPGship_data[j,"TARGET LOCATION"]
    if TPGship_data[j,"TARGET TYPHOON"] != 0:
        target = "Typhoon " + str(target)
    else:
        target = str(target)
    
    #船速取得
    speed = str(format(TPGship_data[j,"SHIP SPEED[kt]"], '.1f'))

    #状態表示　移動・発電・待機
    if (TPGship_data[j,"SHIP SPEED[kt]"] == 0)&(TPGship_data[j,"TIMESTEP POWER GENERATION[Wh]"] == 0) & (TPGship_data[j,"TIMESTEP POWER CONSUMPTION[Wh]"] == 0):
        state = "Standby"
        iro = "lime"
    elif TPGship_data[j,"TIMESTEP POWER GENERATION[Wh]"] > 0:
        state = "Power Generation"
        speed = speed + "(prov)"
        iro = "yellow"
    elif (TPGship_data[j,"TIMESTEP POWER CONSUMPTION[Wh]"] > 0) or (TPGship_data[j,"SHIP SPEED[kt]"] > 0):
        state = "Moving"
        iro = "red"
    
    else:
        print("Error")

    #船内蓄電量
    storage_per = str(format(TPGship_data[j,"ONBOARD POWER STORAGE PER[%]"], '.1f'))

    #TPGship関連情報表示
    r = patches.Rectangle(xy=(145, 58), width=40, height=13, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    

    ax.quiver(view_ship_lon, view_ship_lat, 0, 55, color=iro, edgecolor="k", linewidth = 1.0, headwidth=30, headlength=50 , headaxislength=50, width=.02, scale=400.,zorder=4)
    ax.text(ship_text_lon, ship_text_lat+5.2, "TPGship prototype", size=25, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat+2.5, "Target          : "+target, size=18, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat-0.0, "States          : "+state, size=18, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat-2.5, "Speed[kt]    : "+speed, size=18, color="black",zorder=4)
    ax.text(ship_text_lon, ship_text_lat-5.0, "Storage[%]  : "+storage_per, size=18, color="black",zorder=4)


    #日数記録
    date = str(datetime.fromtimestamp(unixtime,UTC))
    r = patches.Rectangle(xy=(145, 54), width=24, height=4, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    ax.text(145.5, view_lat_1 - 0.8*d_view_lat-0.4, date, size=16.5, color="black",zorder=4)

    #台風の凡例表示
    r = patches.Rectangle(xy=(167, 54), width=20, height=4, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    
    ax.plot(view_lon,view_lat_1 - 0.7*d_view_lat,'grey',markersize=30,marker="o",zorder=4)
    ax.text(text_lon, view_lat_1 - 0.7*d_view_lat-0.5, "Typhoon", size=15, color="black",zorder=4)

    #中継貯蔵拠点の凡例表示
    r = patches.Rectangle(xy=(145, 46), width=20, height=8, ec='k', fc='w',zorder=3)
    stbase_state = str(stBASE_data[j,"BRANCH CONDITION"])
    stbase_storage = str(format(stBASE_data[j,"STORAGE PER[%]"], '.1f'))
    ax.add_patch(r)
    ax.plot(147.0, view_lat_1 - 2.4*d_view_lat,'crimson',markersize=30,marker="d",zorder=4)
    ax.text(149.0, view_lat_1 - 1.8*d_view_lat-0.5, "Storage Base", size=20, color="black",zorder=4)
    ax.text(149.0, view_lat_1 - 2.3*d_view_lat-0.5, "States : "+stbase_state, size=14, color="black",zorder=4)
    ax.text(149.0, view_lat_1 - 2.8*d_view_lat-0.5, "Storage[%]  : "+stbase_storage, size=14, color="black",zorder=4)

    #供給拠点の凡例表示
    ax.plot(147.0, view_lat_1 - 3.6*d_view_lat,'crimson',markersize=20,marker="*",zorder=4)
    ax.text(149.0, view_lat_1 - 3.6*d_view_lat-0.5, "Supply Base", size=15, color="black",zorder=4)

    #運搬船の凡例表示
    r = patches.Rectangle(xy=(165, 46), width=20, height=8, ec='k', fc='w',zorder=3)
    ax.add_patch(r)
    spship1_storage = str(format(spSHIP1_data[j,"STORAGE PER[%]"], '.1f'))
    spship2_storage = str(format(spSHIP2_data[j,"STORAGE PER[%]"], '.1f'))
    ax.quiver(166.5, 51, 0, 15, color="navy", edgecolor="k", linewidth = 0.25, headwidth=6, headlength=10 , headaxislength=10, width=.02, scale=400.,zorder=4)
    ax.text(168.0, view_lat_1 - 1.7*d_view_lat-0.5, "SupportSHIP1", size=15, color="black",zorder=4)
    ax.text(168.0, view_lat_1 - 2.2*d_view_lat-0.5, "Storage[%]  : "+spship1_storage, size=13, color="black",zorder=4)
    ax.quiver(166.5, 46.7, 0, 15, color="purple", edgecolor="k", linewidth = 0.25, headwidth=6, headlength=10 , headaxislength=10, width=.02, scale=400.,zorder=4)
    ax.text(168.0, view_lat_1 - 3.1*d_view_lat-0.5, "SupportSHIP2", size=15, color="black",zorder=4)
    ax.text(168.0, view_lat_1 - 3.6*d_view_lat-0.5, "Storage[%]  : "+spship2_storage, size=13, color="black",zorder=4)


    #運搬船1の作図
    spship1_lat = spSHIP1_data[j,"LAT"]
    spship1_lon = spSHIP1_data[j,"LON"]
    if j == 0:
        u = 0
        v = 1
    else:
        spship1_lat_2 = spSHIP1_data[j-1,"LAT"]
        spship1_lon_2 = spSHIP1_data[j-1,"LON"]
        u = (spship1_lon - spship1_lon_2)
        v = (spship1_lat - spship1_lat_2)
        if (u == 0) & (v == 0):
            u = 0
            v = 1


    size = 10
    
    vec_size = (np.sqrt(u**2 + v**2))
    ax.quiver(spship1_lon, spship1_lat, u/vec_size*size, v/vec_size*size, color="navy", edgecolor="k", linewidth = 0.25, headlength=6 , headaxislength=6, width=.004, scale=400.,zorder=4.5)
    
    #運搬船2の作図
    spship2_lat = spSHIP2_data[j,"LAT"]
    spship2_lon = spSHIP2_data[j,"LON"]
    if j == 0:
        u = 0
        v = 1
    else:
        spship2_lat_2 = spSHIP2_data[j-1,"LAT"]
        spship2_lon_2 = spSHIP2_data[j-1,"LON"]
        u = (spship2_lon - spship2_lon_2)
        v = (spship2_lat - spship2_lat_2)
        if (u == 0) & (v == 0):
            u = 0
            v = 1


    size = 10
    
    vec_size = (np.sqrt(u**2 + v**2))
    ax.quiver(spship2_lon, spship2_lat, u/vec_size*size, v/vec_size*size, color="purple", edgecolor="k", linewidth = 0.25, headlength=6 , headaxislength=6, width=.004, scale=400.,zorder=4.5)
    



    #TPGshipの作図
    ship_lat = TPGship_data[j,"TPGSHIP LAT"]
    ship_lon = TPGship_data[j,"TPGSHIP LON"]
    if j == 0:
        u = 0
        v = 1
    else:
        ship_lat_2 = TPGship_data[j-1,"TPGSHIP LAT"]
        ship_lon_2 = TPGship_data[j-1,"TPGSHIP LON"]
        u = (ship_lon - ship_lon_2)
        v = (ship_lat - ship_lat_2)
        if (u == 0) & (v == 0):
            u = 0
            v = 1


    size = 10
    
    vec_size = (np.sqrt(u**2 + v**2))
    ax.quiver(ship_lon, ship_lat, u/vec_size*size, v/vec_size*size, color=iro, edgecolor="k", linewidth = 0.25, headlength=6 , headaxislength=6, width=.004, scale=400.,zorder=5)
    

    #j = j_ori

    plt.savefig("png_Folder2/draw" + str(j) + ".png")
    plt.close(fig)

    im = Image.open("png_Folder2/draw" + str(j) + ".png")

    #環境ごとに調整
    im_crop = im.crop((150, 250, 1080, 1370))
    im_crop.save("png_Folder2/draw" + str(j) + ".png", quality=100)


KeyboardInterrupt: 